# 2: Replicate 2-layer Additive Risk Model
2018 competition was won by Rudin et al and here's their model results
http://dukedatasciencefico.cs.duke.edu/models/

In [2]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.sparse
import os

# import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import tensorflow.keras.backend as K

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

print('TF version:',tf.__version__)


TF version: 1.15.0


In [36]:
cwd = os.getcwd()
print(cwd)
os.chdir(cwd)

C:\Users\brizio\Documents\PythonNB\FICOchallenge


## Load prepped data
- Normal Scaling of numeric variables (prep_option = 1)
- Binning (following Rudin) and one hot encoding (prep_option = 2)
- Binning and applying WOE, calculating WOE on Rudin's bins (prep_option = 3)
- Binning and applying WOE, following Rudin (prep_option = 4)

In [1]:
prep_option = 4

In [62]:
if prep_option == 1:
    data_path = "Data/Scaled_data.csv"
if prep_option == 2:
    data_path = "Data/Bin_Encoded_data_v2.csv"
if prep_option == 3:
    data_path = "Data/WOE_data.csv"
if prep_option == 4:
    data_path = "Data/WOE_Rud_data.csv"
    
ori_path = "Data/heloc_dataset_v1.csv"

CLASS = 'RiskPerformance' 

data = pd.read_csv(ori_path)
X1 = pd.read_csv(data_path)
y = pd.read_csv("Data/y_data.csv")

print('Target: Bad (y=1)')
class_names = sorted(y[CLASS].unique(),  reverse=True)
print(y[CLASS].value_counts())
y_onehot = pd.get_dummies(y[CLASS])[['Bad']]
print(np.array(np.unique(y_onehot, return_counts=True)).T)

print('X shape:',X1.shape)
X1.head()

Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 23)


,ExternalRiskEstimate_bin_WOE,MSinceOldestTradeOpen_bin_WOE,MSinceMostRecentTradeOpen_bin_WOE,AverageMInFile_bin_WOE,NumSatisfactoryTrades_bin_WOE,NumTrades60Ever2DerogPubRec_bin_WOE,NumTrades90Ever2DerogPubRec_bin_WOE,NumTotalTrades_bin_WOE,NumTradesOpeninLast12M_bin_WOE,PercentTradesNeverDelq_bin_WOE,...,PercentInstallTrades_bin_WOE,NetFractionInstallBurden_bin_WOE,NumInstallTradesWBalance_bin_WOE,MSinceMostRecentInqexcl7days_bin_WOE,NumInqLast6M_bin_WOE,NumInqLast6Mexcl7days_bin_WOE,NetFractionRevolvingBurden_bin_WOE,NumRevolvingTradesWBalance_bin_WOE,NumBank2NatlTradesWHighUtilization_bin_WOE,PercentTradesWBalance_bin_WOE
0,1.799,0.086,0.083,0.269,0.166,0.952,-0.021,-0.097,-0.021,1.012,...,-0.503,0.047,0.242,1.223,-0.047,-0.051,-0.088,0.034,-0.601,-0.130
1,1.799,0.549,0.083,1.238,1.999,0.952,-0.053,0.535,-0.021,-0.147,...,0.161,0.047,0.256,1.223,-0.047,-0.051,-0.739,-0.188,0.601,-0.982
2,1.017,0.549,0.083,1.238,0.539,-0.021,-0.021,0.535,0.428,-0.147,...,-0.503,0.147,0.242,1.223,0.170,0.021,0.633,-0.263,-0.601,0.203
3,1.017,0.086,0.083,0.269,-0.086,-0.021,-0.021,-0.377,0.287,0.366,...,-0.145,0.363,0.313,1.223,0.471,0.021,0.633,-0.150,0.541,0.772
4,-1.094,-0.148,0.000,0.000,0.539,-0.021,-0.021,0.116,-0.021,-0.147,...,-0.620,0.363,0.242,1.223,-0.047,-0.051,0.633,-0.188,-0.601,0.203


## Define Feature Groups (Subscales)

In [184]:
def get_feat_map(x_onehot,X):
    """
        Define tuples of columns with corresponding manually defined groupings. 
        The groupings below are from Rudin et al paper winning the 2018 FICO challenge.
        Output a dataframe of original columns and group classes ready for manipulation.
    """
    
    feat_map = [
     (X.columns.values[0],'ExternalRiskEstimate'),

     (X.columns.values[1],'TradeOpenTime'),
     (X.columns.values[2],'TradeOpenTime'),
     (X.columns.values[3],'TradeOpenTime'),

     (X.columns.values[4],'NumSatisfactoryTrades'),

     (X.columns.values[5],'TradeFrequency'),
     (X.columns.values[6],'TradeFrequency'),
     (X.columns.values[7],'TradeFrequency'),
     (X.columns.values[8],'TradeFrequency'),

     (X.columns.values[9],'Delinquency'),
     (X.columns.values[10],'Delinquency'),
     (X.columns.values[11],'Delinquency'),
     (X.columns.values[12],'Delinquency'),

     (X.columns.values[13],'Installment'),
     (X.columns.values[14],'Installment'),
     (X.columns.values[15],'Installment'),

     (X.columns.values[16],'Inquiry'),
     (X.columns.values[17],'Inquiry'),
     (X.columns.values[18],'Inquiry'),

     (X.columns.values[19],'RevolvingBalance'),
     (X.columns.values[20],'RevolvingBalance'),

     (X.columns.values[21],'Utilization'),

     (X.columns.values[22],'TradeWBalance')

    ]
    
    nme = x_onehot.columns.values
    deg = ['_'.join(w.split('_')[:-2]) for w in nme]
#     print(deg)
    feat_map
    feat_map_df = pd.DataFrame(feat_map, columns=['Feature','Group'])
  
    return feat_map_df

In [185]:
feat_map = get_feat_map(X1,X1)
# print(pd.DataFrame(X1.columns.values))
feat_map

,Feature,Group
0,ExternalRiskEstimate_bin_WOE,ExternalRiskEstimate
1,MSinceOldestTradeOpen_bin_WOE,TradeOpenTime
2,MSinceMostRecentTradeOpen_bin_WOE,TradeOpenTime
3,AverageMInFile_bin_WOE,TradeOpenTime
4,NumSatisfactoryTrades_bin_WOE,NumSatisfactoryTrades
5,NumTrades60Ever2DerogPubRec_bin_WOE,TradeFrequency
6,NumTrades90Ever2DerogPubRec_bin_WOE,TradeFrequency
7,NumTotalTrades_bin_WOE,TradeFrequency
8,NumTradesOpeninLast12M_bin_WOE,TradeFrequency
9,PercentTradesNeverDelq_bin_WOE,Delinquency


## Load 2nd Layer weights and biases from Rudin et al model

In [186]:
weights_and_biases_2ndL = pd.read_csv('2layer_weights.csv')
weights_and_biases_2ndL

,Group,Bias,Risk Score,Weight,Points
0,ExternalRiskEstimate,-0.290,0.819,1.593,1.304667
1,TradeOpenTime,-0.551,0.523,2.468,1.290764
2,NumSatisfactoryTrades,-0.097,0.517,2.273,1.175141
3,TradeFrequency,-0.042,0.450,0.358,0.161100
4,Delinquency,-0.237,0.799,2.470,1.973530
5,Installment,0.101,0.561,1.175,0.659175
6,Inquiry,-0.808,0.639,2.994,1.913166
7,RevolvingBalance,0.135,0.803,1.877,1.507231
8,Utilization,0.366,0.442,1.119,0.494598
9,TradeWBalance,0.227,0.731,0.214,0.156434


Merge it onto the groups list

In [187]:
feat_map = feat_map.merge(weights_and_biases_2ndL, how='left')[['Feature','Group', 'Bias', 'Weight']]
feat_map

,Feature,Group,Bias,Weight
0,ExternalRiskEstimate_bin_WOE,ExternalRiskEstimate,-0.290,1.593
1,MSinceOldestTradeOpen_bin_WOE,TradeOpenTime,-0.551,2.468
2,MSinceMostRecentTradeOpen_bin_WOE,TradeOpenTime,-0.551,2.468
3,AverageMInFile_bin_WOE,TradeOpenTime,-0.551,2.468
4,NumSatisfactoryTrades_bin_WOE,NumSatisfactoryTrades,-0.097,2.273
5,NumTrades60Ever2DerogPubRec_bin_WOE,TradeFrequency,-0.042,0.358
6,NumTrades90Ever2DerogPubRec_bin_WOE,TradeFrequency,-0.042,0.358
7,NumTotalTrades_bin_WOE,TradeFrequency,-0.042,0.358
8,NumTradesOpeninLast12M_bin_WOE,TradeFrequency,-0.042,0.358
9,PercentTradesNeverDelq_bin_WOE,Delinquency,-0.237,2.470


## Load the WOE by bin definition

This will be used to map each account characteristic onto their correspondent points

In [188]:
csv = "./WOE_bins_Rud.csv"
# csv = "synth_nonlinear.csv"
WOE_df = pd.read_csv(csv)
print(WOE_df.shape)
WOE_var = 'WOE_Rud'
WOE_df['Feature'] = WOE_df['Feature'].astype(str)+"_WOE"
WOE_df = WOE_df.merge(feat_map, how='left')[['Group', 'Feature', 'Bin', WOE_var, 'Bias', 'Weight']]
WOE_df[WOE_df['Feature']=='AverageMInFile_bin_WOE']
WOE_df = round(WOE_df,3)
# print(WOE_df.shape)


(165, 12)


## Load an example from the data

In [241]:
# idx = 1468 # Demo 1
# idx = 131 # Demo 2
idx = 3291 # Demo 3
data.loc[idx]

RiskPerformance                       Good
ExternalRiskEstimate                    92
MSinceOldestTradeOpen                  372
MSinceMostRecentTradeOpen               10
AverageMInFile                         176
NumSatisfactoryTrades                   25
NumTrades60Ever2DerogPubRec              0
NumTrades90Ever2DerogPubRec              0
PercentTradesNeverDelq                 100
MSinceMostRecentDelq                    -7
MaxDelq2PublicRecLast12M                 7
MaxDelqEver                              8
NumTotalTrades                          68
NumTradesOpeninLast12M                   1
PercentInstallTrades                    28
MSinceMostRecentInqexcl7days            -8
NumInqLast6M                             0
NumInqLast6Mexcl7days                    0
NetFractionRevolvingBurden               3
NetFractionInstallBurden                -8
NumRevolvingTradesWBalance               2
NumInstallTradesWBalance                -8
NumBank2NatlTradesWHighUtilization       0
PercentTrad

## Scoring Function

In [267]:

def AddRiskModel_pred(df, WOE_df, idx):
    x_original = pd.DataFrame(data.loc[idx]).reset_index()
    x_original.columns = ['Feature', str(idx)+'_value']
    x_to_pred = pd.DataFrame(round(df.loc[idx],3)).reset_index()
    x_to_pred[str(idx)] = np.repeat('yes',len(df.loc[idx]))
    x_to_pred.columns = ['Feature', WOE_var, str(idx)]
    WOE_df_complete = WOE_df.merge(x_to_pred, how='right')
    WOE_df_complete
    WOE_agg = WOE_df_complete.groupby('Group').agg({WOE_var:'sum','Bias':'mean','Weight':'mean'})
    WOE_agg['P_group'] = 1/(1+np.exp(-(WOE_agg[WOE_var ] + WOE_agg['Bias'])))
    WOE_agg['Points'] = WOE_agg['P_group']*WOE_agg['Weight']
    WOE_agg
    overall_bias = -8.495
    total = np.sum(WOE_agg['Points'])
    logodds = total + overall_bias
    # print(logodds)
    p_out = 1/(1+np.exp(-logodds))
    return round(p_out,3)

print("Account {} has probability of bad = {:.1%}".format(idx,AddRiskModel_pred(X1,WOE_df,idx)))

Account 3291 has probability of bad = 4.9%


## Predict the whole dataset

In [ ]:
pred_array = []
for i in tqdm(range(len(X1)):
    pred_array.append(AddRiskModel_pred(X1,WOE_df,i))

## Assess Accuracy

In [208]:
from sklearn.metrics import classification_report,accuracy_score, roc_auc_score

y_pred = np.array([int(i > .5) for i in pred_array])
print(classification_report(y_onehot,y_pred))
print('Accuracy:',accuracy_score(y_onehot, y_pred))
print('AUC:',roc_auc_score(y_onehot, pred_array))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72      5000
           1       0.74      0.73      0.74      5459

    accuracy                           0.73     10459
   macro avg       0.73      0.73      0.73     10459
weighted avg       0.73      0.73      0.73     10459

Accuracy: 0.7269337412754565
AUC: 0.7972186847407952


### On (my) test set only

In [268]:
X_train, X_test, y_train, y_test = train_test_split(X1, y_onehot, test_size = .25, random_state = 2020, shuffle = True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y_pred_test = y_pred[y_test.index]
print(classification_report(y_test,y_pred_test))
# print('Accuracy:',classification_report(y_test,y_pred_test,output_dict=True)['accuracy'])
print('Accuracy:',accuracy_score(y_test, y_pred_test))
print('AUC:',roc_auc_score(y_test, np.array(pred_array)[y_test.index]))

(7844, 23) (2615, 23) (7844, 1) (2615, 1)
              precision    recall  f1-score   support

           0       0.69      0.73      0.71      1224
           1       0.75      0.71      0.73      1391

    accuracy                           0.72      2615
   macro avg       0.72      0.72      0.72      2615
weighted avg       0.72      0.72      0.72      2615

Accuracy: 0.7181644359464627
AUC: 0.7857383248051197
